[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/PoseIQ/BiomechPythonAI_Guide/blob/main/PoseIQ_Chapter0_C3D_ColabNotebook_UPDATED.ipynb)

# Chapter 0: Getting Started with C3D Files in Python and Colab

**Ebook**: *A Hands-On Guide to Biomechanics Data Analysis with Python and AI*  
**Author**: Dr. Hossein Mokhtarzadeh  
**Powered by**: PoseIQ™

Welcome to Chapter 0. This notebook will walk you through loading and visualizing biomechanics C3D files in Google Colab.

We’ve been developing **PoseIQ™** tools for physio, human movement, and rehab using just a camera. This ebook and its companion course on **[Udemy](https://www.udemy.com/user/hossein-mokhtarzadeh/)** are designed to teach you by doing — no prior experience with C3D, Python, or Colab needed.

📌 **Quick Start**: Just click `Runtime > Run all`, and you're good to go.  
You'll download a sample C3D dataset, install the needed tools, load and plot your first force plate signal.

🔗 [PoseIQ™ on LinkedIn](https://www.linkedin.com/company/poseiq/)  
🔗 [Demo Page](https://poseiq.com/demos)  
🔗 [poseiq.com](https://poseiq.com)


# Chapter 0: Getting Started with C3D Files in Python and Colab

**Ebook**: *A Hands-On Guide to Biomechanics Data Analysis with Python and AI*  
**Author**: Dr. Hossein Mokhtarzadeh  
**Powered by**: PoseIQ™

Welcome to Chapter 0. This notebook will walk you through loading and visualizing biomechanics C3D files in Google Colab.

We’ve been developing **PoseIQ™** tools for physio, human movement, and rehab using just a camera. This ebook and its companion course on **Udemy** are designed to teach you by doing — no prior experience with C3D, Python, or Colab needed.

📌 **Quick Start**: Just click `Runtime > Run all`, and you're good to go.  
You'll download a sample C3D dataset, install the needed tools, load and plot your first force plate signal.

🔗 [PoseIQ™ on LinkedIn](https://www.linkedin.com/company/poseiq/)  
🔗 [Demo Page](https://poseiq.com/demos)  
🔗 [poseiq.com](https://poseiq.com)


In [ ]:
!pip install ezc3d pandas numpy

In [ ]:
# Step 1: Download and unzip
import os, urllib.request, zipfile, sys, subprocess

zip_url = "https://c3d.org/data/Sample01.zip"
zip_path = "Sample01.zip"
if not os.path.exists("c3d_samples"):
    urllib.request.urlretrieve(zip_url, zip_path)
    with zipfile.ZipFile(zip_path, "r") as zf:
        zf.extractall("c3d_samples")

# Step 2: Install ezc3d if missing
try:
    import ezc3d  # noqa
except ImportError:
    subprocess.run([sys.executable, "-m", "pip", "install", "ezc3d"], check=True)
import ezc3d

# Step 3: find all C3D files
c3d_files = []
for root, dirs, files in os.walk("c3d_samples"):
    for f in files:
        if f.lower().endswith(".c3d"):
            c3d_files.append(os.path.join(root, f))

print("Found C3D files:", c3d_files)

# Step 4: load the first file that ezc3d supports and that has analogs
loaded = None
loaded_path = None
errors = {}

for path in c3d_files:
    try:
        c = ezc3d.c3d(path)
        # check if any analog samples exist
        analogs = c["data"]["analogs"]
        if analogs.size == 0:
            # keep as fallback only if we never find one with analogs
            if loaded is None:
                loaded = c
                loaded_path = path
            continue
        loaded = c
        loaded_path = path
        break
    except Exception as e:
        errors[path] = str(e)
        continue

if loaded is None:
    raise RuntimeError(
        "Could not load a supported C3D. Last errors:\n" + "\n".join(f"{k}: {v}" for k, v in errors.items())
    )

print("Loaded:", loaded_path)

# Step 5: plot the first analog channel if present
import numpy as np
import matplotlib.pyplot as plt

params = loaded["parameters"]
has_analog = "ANALOG" in params and "RATE" in params["ANALOG"]

if has_analog and loaded["data"]["analogs"].size > 0:
    analog_labels = []
    if "LABELS" in params["ANALOG"]:
        analog_labels = params["ANALOG"]["LABELS"]["value"]

    analogs = loaded["data"]["analogs"]  # shape: (n_chan, n_subframes, n_frames)
    n_chan, n_sub, n_fr = analogs.shape
    idx = 0  # first channel

    # flatten over time with frames then subframes
    signal = analogs[idx].T.reshape(-1)  # shape (n_frames, n_subframes) -> flatten to 1D
    rate = float(params["ANALOG"]["RATE"]["value"][0])
    t = np.arange(signal.size) / rate

    label = analog_labels[idx] if analog_labels and idx < len(analog_labels) else f"Channel {idx}"

    plt.figure()
    plt.plot(t, signal)
    plt.title(f"Analog Channel: {label}")
    plt.xlabel("Time (s)")
    plt.ylabel("Signal")
    plt.grid(True)
    plt.show()
else:
    print("No analog channels found in:", loaded_path)
    if errors:
        print("Some files were skipped due to reader limits, for example:")
        for k, v in list(errors.items())[:3]:
            print(" ", k, "->", v)


In [ ]:
# Extract analog data
analog_data = c['data']['analogs']

# Extract analog labels
analog_labels = c['parameters']['ANALOG']['LABELS']['value']

print("Analog data shape:", analog_data.shape)
print("Analog labels:", analog_labels)

In [ ]:
import pandas as pd

# Transpose analog_data to have channels as columns and samples as rows
# The first dimension is a batch dimension of size 1, so we can ignore it
analog_data_transposed = analog_data[0].T

# Create a pandas DataFrame
df_analog = pd.DataFrame(analog_data_transposed)

# Assign analog labels as column names
df_analog.columns = analog_labels

# Display the head of the DataFrame
display(df_analog.head())

In [ ]:
display(df_analog.head())

In [ ]:
def plot_analog_channel(df, column_name):
    """
    Plots a specified analog channel from a DataFrame against time.

    Args:
        df (pd.DataFrame): The DataFrame containing analog data.
        column_name (str): The name of the column (analog channel) to plot.
    """
    # Calculate the time vector
    analog_rate = c['parameters']['ANALOG']['RATE']['value']
    t = np.arange(df.shape[0]) / analog_rate

    # Create the plot
    plt.figure(figsize=(10, 6))
    plt.plot(t, df[column_name])

    # Set title and labels
    plt.title(f"Analog Channel: {column_name}")
    plt.xlabel("Time (s)")
    plt.ylabel("Signal")

    # Add grid
    plt.grid(True)

    # Display the plot
    plt.show()

In [ ]:
# Step 1: Print available columns
print("\nAvailable Analog Channels:")
for col in df_analog.columns:
    print(f"- {col}")

# Step 2 & 3: Prompt user and implement error handling
selected_column = None
while selected_column not in df_analog.columns:
    selected_column = input("\nEnter the name of the analog channel to plot: ")
    if selected_column not in df_analog.columns:
        print("Invalid column name. Please try again.")

print(f"\nYou selected: {selected_column}")

In [ ]:
plot_analog_channel(df_analog, selected_column)

## Summary:

### Data Analysis Key Findings

*   The zip file "/content/Sample01.zip" was successfully extracted to "/content/unzipped_c3d_files".
*   Six C3D files were identified in the extracted directory.
*   Analog data, including GRF information, was successfully extracted from all six C3D files.
*   GRF data (columns starting with 'Force.F') was isolated from the analog data for each file.
*   The GRF data from all files was combined into a single DataFrame, including a 'File' column to identify the source of each data point.
*   A single plot was generated showing the 'Force.Fz' component (vertical GRF) over time for each of the six files.

### Insights or Next Steps

*   The generated plot provides a visual comparison of the vertical GRF profiles across different C3D files, which could be useful for identifying variations in gait or movement patterns.
*   Further analysis could involve calculating peak GRF values, impulse, or other relevant metrics from the extracted data.
